In [6]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import json

In [11]:
# Creación del manipulador del navegador
options = webdriver.ChromeOptions()
driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=options)
driver.implicitly_wait(3)

In [12]:
# Apertura de la página
url = 'https://gamewith.net/genshin-impact/'
driver.get(url)
driver.implicitly_wait(5)

In [13]:
# Eliminación del banner de los ads

driver.find_element(By.CLASS_NAME, "fc-cta-consent").click()

# <button class="fc-button fc-cta-consent fc-primary-button" role="button" aria-label="Consent" tabindex="0"><div class="fc-button-background"></div><p class="fc-button-label">Consent</p></button>

In [14]:
# Eliminación del banner de las cookies 
driver.find_element(By.CLASS_NAME, "js-cookie-consent-btn").click()

In [17]:
# Tabla de 5 estrellas
link_list = []

for i in range(2, 11):
    fila = f'/html/body/div[3]/div[5]/div[1]/div/div[1]/div[5]/div[32]/table[1]/tbody/tr[{i}]'
    for j in range(1, 5):
        character = driver.find_element(By.XPATH, f'{fila}/td[{j}]/span/a')
        link = character.get_attribute('href')
        if "gamewith.net" in link:
            link_list.append(link)
            
# /html/body/div[3]/div[5]/div[1]/div/div[1]/div[5]/div[32]/table[1]/tbody/tr[2]/td[1]/span/a

In [19]:
# Tabla de 4 estrellas
for i in range(12,20):
    fila = f'/html/body/div[3]/div[5]/div[1]/div/div[1]/div[5]/div[32]/table[1]/tbody/tr[{i}]'
    for j in range(1,5):
        character = driver.find_element(By.XPATH, f'{fila}/td[{j}]/span/a')
        link = character.get_attribute('href')
        if "gamewith.net" in link:
            link_list.append(link)
                             
# /html/body/div[3]/div[5]/div[1]/div/div[1]/div[5]/div[32]/table[1]/tbody/tr[12]
        
display(link_list)

['https://gamewith.net/genshin-impact/article/show/23420',
 'https://gamewith.net/genshin-impact/article/show/33663',
 'https://gamewith.net/genshin-impact/article/show/29597',
 'https://gamewith.net/genshin-impact/article/show/22643',
 'https://gamewith.net/genshin-impact/article/show/22837',
 'https://gamewith.net/genshin-impact/article/show/24635',
 'https://gamewith.net/genshin-impact/article/show/22922',
 'https://gamewith.net/genshin-impact/article/show/24371',
 'https://gamewith.net/genshin-impact/article/show/33787',
 'https://gamewith.net/genshin-impact/article/show/29670',
 'https://gamewith.net/genshin-impact/article/show/22351',
 'https://gamewith.net/genshin-impact/article/show/28610',
 'https://gamewith.net/genshin-impact/article/show/25134',
 'https://gamewith.net/genshin-impact/article/show/23418',
 'https://gamewith.net/genshin-impact/article/show/22642',
 'https://gamewith.net/genshin-impact/article/show/23791',
 'https://gamewith.net/genshin-impact/article/show/29489

In [20]:
len(link_list)

68

In [21]:
with open('link_list.json', 'w') as f:
    json.dump(link_list, f)

In [16]:
# Test opening all the links
for link in link_list:
    driver.get(link)
    driver.implicitly_wait(3)

In [24]:
# Abrir el json de premium_teams
with open('../data/characters_premium.json', 'r') as f:
    teams = json.load(f)

In [32]:
# CODIGO BUENO

# Crear una lista para almacenar todos los datos
f2p_teams = []

# Iterar a través de la lista de enlaces
for link in link_list:
    driver.get(link)
    driver.implicitly_wait(3)

    # Obtener el nombre del héroe
    nombre = driver.find_element(
        By.XPATH, '/html/body/div[7]/div/div[1]/div[1]/div[1]/div[1]/h1')
    nombre = nombre.text.replace("Genshin Impact",
                                 "").replace("Best Build & Weapon",
                                             "").strip()

    # Obtener los miembros del equipo free-to-play
    # Encontrar el elemento que contiene el texto "Free To Play Friendly Party"
    try:    
        try:
            f2p_team_heading = driver.find_element_by_xpath(
                "//h3[contains(text(), 'Free To Play Friendly Party') or contains(text(), 'Free-To-Play Party') or contains(text(), 'F2P Friendly Team')]"
            )
        except:
            f2p_team_heading = driver.find_element_by_xpath(
                "//h4[contains(text(), 'Free To Play Friendly Party') or contains(text(), 'Free-To-Play Party') or contains(text(), 'F2P Friendly Team')]"
            )

        # Navegar al elemento de tabla que está justo debajo
        f2p_team_table = f2p_team_heading.find_element(By.XPATH,
                                                       "following-sibling::table")

        # Coger la fila con los nombres de los characters
        roles = f2p_team_table.find_elements(By.TAG_NAME, "tr")[0]
        compis = f2p_team_table.find_elements(By.TAG_NAME, "tr")[1]

        # Extraer los roles
        f2p_team_roles = [
            rol.text for rol in roles.find_elements_by_css_selector('th')
        ]

        print(f2p_team_roles)

        # Extraer los nombres de los team members
        f2p_team = []
        for i in range(0, 4):
            f2p_team.append(compis.text.splitlines()[i])

        print(f2p_team)

        # Obtener los detalles del equipo premium
        detalles = f2p_team_table.find_elements(By.TAG_NAME,
                                                "tr")[3].text.splitlines()
    except:
        f2p_team_roles = []
        f2p_team = []
        detalles = []

    # Crear un diccionario con la información del héroe
    f2p_team = {
        "id": nombre.lower().replace(" ", "-"),
        "name": nombre,
        "f2p_team_roles": f2p_team_roles,
        "f2p_team": f2p_team,
        "f2p_team_details": detalles
    }

    # Agregar el diccionario a la lista de datos
    f2p_teams.append(f2p_team)

    driver.implicitly_wait(1)

['Main DPS', 'Sub-DPS', 'Sub-DPS', 'Support']
['Xiangling', 'Xingqiu', 'Albedo', 'Fischl']
['Main', 'Sub-DPS', 'Resonance', 'Support']
['Alhaitham', 'Fischl', 'Collei', 'Barbara']
['DPS', 'Reaction', 'Support', 'Healing']
['Aloy', 'Xiangling', 'Anemo Traveler', 'Barbara']
['Support', 'Main DPS', 'Sub DPS', 'Support']
['Anemo Traveler', 'Lisa', 'Xiangling', 'Barbara']
['Main DPS', 'Sub-DPS', 'Sub-DPS', 'Sub-DPS / Healer']
['Ayaka', 'Xiangling', 'Xingqiu', 'Diona']
['DPS', 'Combo', 'Heal', 'Support']
['Ayato', 'Xiangling', 'Barbara', 'Dendro Traveler']
['Main DPS', 'Sub-DPS', 'Sub-DPS', 'Support']
['Childe', 'Xiangling', 'Sucrose', 'Bennett']
['DPS', 'Sub-DPS', 'Reaction', 'Heal']
['Cyno', 'Fischl', 'Dendro Traveler', 'Barbara']
['Main', 'Sub-DPS', 'Support', 'Healer']
['Dehya', 'Collei', 'Sucrose', 'Barbara']
['Sub-DPS', 'Support', 'DPS', 'Support']
['Dendro Traveler', 'Fischl', 'Lisa', 'Barbara']
['Main DPS', 'Sub-DPS', 'Sub-DPS', 'Support']
['Diluc', 'Xingqiu', 'Fischl', 'Sucrose']
['

NameError: name 'characters_data' is not defined

In [34]:
# Convertir la lista de datos en formato JSON y guardarla en un archivo
with open('../data/characters_f2p.json', 'w') as outfile:
    json.dump(f2p_teams, outfile)

In [35]:
with open('../data/characters_premium.json', 'r') as f:
    premium_team = json.load(f)

In [36]:
ids = []
for character in premium_team:
    ids.append(character["id"])
len(ids)

68

In [38]:
ids2 = []
for character in f2p_teams:
    ids2.append(character["id"])
len(ids2)

68

In [39]:
# Find items that are only in ids
only_in_ids = set(ids) - set(ids2)
print("Items only in ids:", only_in_ids)

# Find items that are only in ids_team2
only_in_ids2 = set(ids2) - set(ids)
print("Items only in ids2:", only_in_ids2)

Items only in ids: {'wanderer', 'arataki-itto', 'tartaglia', 'sangonomiya-kokomi', 'kamisato-ayaka', 'kaedehara-kazuha'}
Items only in ids_team: {'itto', 'scaramouche-(wanderer)', 'kazuha', 'ayaka', 'childe-(tartaglia)', 'kokomi'}


In [41]:
for char in f2p_teams:
        if char['id'] == 'childe-(tartaglia)':
            char['id'] = 'tartaglia'
        elif char['id'] == 'scaramouche-(wanderer)':
            char['id'] = 'wanderer'
        elif char['id'] == 'itto':
            char['id'] = 'arataki-itto'
        elif char['id'] == 'ayaka':
            char['id'] = 'kamisato-ayaka'
        elif char['id'] == 'kazuha':
            char['id'] = 'kaedehara-kazuha'
        elif char['id'] == 'kokomi':
            char['id'] = 'sangonomiya-kokomi'

In [43]:
ids2 = []
for character in f2p_teams:
    ids2.append(character["id"])
len(ids2)

68

In [44]:
# Find items that are only in ids
only_in_ids = set(ids) - set(ids2)
print("Items only in ids:", only_in_ids)

# Find items that are only in ids_team2
only_in_ids2 = set(ids2) - set(ids)
print("Items only in ids2:", only_in_ids2)

Items only in ids: set()
Items only in ids2: set()


In [45]:
for character in f2p_teams:
    character["f2p_team_ids"] = [s.lower().replace(' ', '-') for s in character["f2p_team"]]
f2p_teams

[{'id': 'albedo',
  'name': 'Albedo',
  'f2p_team_roles': ['Main DPS', 'Sub-DPS', 'Sub-DPS', 'Support'],
  'f2p_team': ['Xiangling', 'Xingqiu', 'Albedo', 'Fischl'],
  'f2p_team_details': ['- A relatively free-to-play team comp for Albedo',
   "- Full team includes continuous damage, good to proc Albedo's skill"],
  'f2p_team_ids': ['xiangling', 'xingqiu', 'albedo', 'fischl']},
 {'id': 'alhaitham',
  'name': 'Alhaitham',
  'f2p_team_roles': ['Main', 'Sub-DPS', 'Resonance', 'Support'],
  'f2p_team': ['Alhaitham', 'Fischl', 'Collei', 'Barbara'],
  'f2p_team_details': ['- This team is a Spread-Hyperbloom team that is composed of easily obtained 4 star characters.',
   '-Barbara will be the healer and off-field Hydro.',
   '- Fischl for off-field Electro.',
   '- Collei for resonance buff.',
   'Alhaitham for main DPS.'],
  'f2p_team_ids': ['alhaitham', 'fischl', 'collei', 'barbara']},
 {'id': 'aloy',
  'name': 'Aloy',
  'f2p_team_roles': ['DPS', 'Reaction', 'Support', 'Healing'],
  'f2p_te

In [48]:
ids_team = []
for character in f2p_teams:
    for char in character['f2p_team_ids']:
        ids_team.append(char)
ids_team

['xiangling',
 'xingqiu',
 'albedo',
 'fischl',
 'alhaitham',
 'fischl',
 'collei',
 'barbara',
 'aloy',
 'xiangling',
 'anemo-traveler',
 'barbara',
 'anemo-traveler',
 'lisa',
 'xiangling',
 'barbara',
 'ayaka',
 'xiangling',
 'xingqiu',
 'diona',
 'ayato',
 'xiangling',
 'barbara',
 'dendro-traveler',
 'childe',
 'xiangling',
 'sucrose',
 'bennett',
 'cyno',
 'fischl',
 'dendro-traveler',
 'barbara',
 'dehya',
 'collei',
 'sucrose',
 'barbara',
 'dendro-traveler',
 'fischl',
 'lisa',
 'barbara',
 'diluc',
 'xingqiu',
 'fischl',
 'sucrose',
 'electro-traveler',
 'xiangling',
 'kaeya',
 'barbara',
 'eula',
 'fischl',
 'diona',
 'lisa',
 'ganyu',
 'barbara',
 'sucrose',
 'diona',
 'geo-traveler',
 'noelle',
 'xiangling',
 'kaeya',
 'hu-tao',
 'xingqiu',
 'sucrose',
 'diona',
 'arataki-itto',
 'xiangling',
 'geo-traveler',
 'noelle',
 'jean',
 'rosaria',
 'xingqiu',
 'sucrose',
 'xiangling',
 'xingqiu',
 'kazuha',
 'bennett',
 'keqing',
 'layla',
 'fischl',
 'diona',
 'klee',
 'xingqiu'

In [49]:
# Find items that are only in ids
only_in_ids = set(ids) - set(ids_team)
print("Items only in ids:", only_in_ids)

# Find items that are only in ids_team2
only_in_ids_team = set(ids_team) - set(ids)
print("Items only in ids_team:", only_in_ids_team)

Items only in ids: {'tartaglia', 'sangonomiya-kokomi', 'kamisato-ayaka', 'kaedehara-kazuha', 'xinyan'}
Items only in ids_team: {'childe', 'ayaka', 'kazuha', 'kokomi'}


In [50]:
for character in f2p_teams:
    for i, char in enumerate(character['f2p_team_ids']):
        if char == 'childe':
            character['f2p_team_ids'][i] = 'tartaglia'
        elif char == 'ayaka':
            character['f2p_team_ids'][i] = 'kamisato-ayaka'
        elif char == 'kazuha':
            character['f2p_team_ids'][i] = 'kaedehara-kazuha'
        elif char == 'kokomi':
            character['f2p_team_ids'][i] = 'sangonomiya-kokomi'

In [52]:
ids_team = []
for character in f2p_teams:
    for char in character['f2p_team_ids']:
        ids_team.append(char)

In [53]:
# Find items that are only in ids
only_in_ids = set(ids) - set(ids_team)
print("Items only in ids:", only_in_ids)

# Find items that are only in ids_team2
only_in_ids_team = set(ids_team) - set(ids)
print("Items only in ids_team:", only_in_ids_team)

Items only in ids: {'xinyan'}
Items only in ids_team: set()


In [54]:
# Convertir la lista final de datos en formato JSON y guardarla en un archivo
with open('../data/characters_f2p.json', 'w') as outfile:
    json.dump(f2p_teams, outfile)